In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('spam.csv', encoding='latin-1')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [6]:
# # حذف الأعمدة غير الضرورية وإعادة تسمية الأعمدة المهمة


# df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], errors='ignore')
# df.columns = ['label', 'text']
# لماذا حدث الخطأ؟
# السبب في حدوث هذا الخطأ هو أن ملف الـ CSV الخاص بـ Kaggle أحياناً يتم قراءته بشكل "فوضوي" 
# بسبب وجود فواصل (Commas) داخل نصوص الرسائل نفسها،
#  مما يجعل Pandas تعتقد أن هناك أعمدة إضافية (Unnamed).

# 2. بدلاً من حذف أعمدة محددة بالاسم، سنأخذ أول عمودين فقط بالترتيب
# هذا يحل مشكلة وجود أعمدة فارغة أو بأسماء غريبة (مثل Unnamed: 2) التي تظهر أحياناً عند تحميل الملف.

df = df.iloc[:, [0, 1]] 
# iloc[:, [0, 1]]: هذه التعليمة السحرية تخبر بايثون: "يا بايثون، لا يهمني ما هي أسماء الأعمدة، فقط أعطني كل الأسطر : وأول عمودين [0, 1]".

# 3. إعادة تسمية الأعمدة لتكون واضحة للكود
df.columns = ['label', 'text']

# تحويل الليبل إلى أرقام (spam = 1, ham = 0)
df['label_num'] = df['label'].map({'spam': 1, 'ham': 0})

print(df.head())

  label                                               text  label_num
0   ham  Go until jurong point, crazy.. Available only ...          0
1   ham                      Ok lar... Joking wif u oni...          0
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...          1
3   ham  U dun say so early hor... U c already then say...          0
4   ham  Nah I don't think he goes to usf, he lives aro...          0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      5572 non-null   object
 1   text       5572 non-null   object
 2   label_num  5572 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


# 2. تنظيف النصوص (Preprocessing)


<!-- قبل تدريب الموديل، يفضل جعل النصوص "بسيطة" ليفهمها الكمبيوتر بشكل أفضل (مثل تحويل كل الحروف لصغيرة وإزالة علامات الترقيم). -->

In [8]:
import re
# هذه المكتبة مخصصة لـ Regular Expressions (التعبيرات النمطية).
#  وهي أداة قوية جداً للبحث عن أنماط معينة داخل النصوص (مثل البحث عن أرقام فقط، أو إيميلات،
#  أو في حالتنا هذه: علامات الترقيم).  
  
def clean_text(text):
    # تحويل النص لحروف صغيرة
    text = text.lower()
    # إزالة علامات الترقيم والرموز الخاصة
    text = re.sub(r'[^\w\s]', '', text)
    return text

df['text_clean'] = df['text'].apply(clean_text)

# 3. تدريب الموديل على آلاف الرسائل

<!-- الآن سنقوم بتقسيم البيانات إلى "بيانات تدريب" و"بيانات اختبار" لنعرف مدى دقة الموديل في الواقع. -->

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

# تقسيم البيانات (80% تدريب، 20% اختبار)
X_train, X_test, y_train, y_test = train_test_split(df['text_clean'], df['label_num'], test_size=0.2, random_state=42)

# تحويل النصوص إلى مصفوفة أرقام
# vectorizer = CountVectorizer()
# لكن هذه الاداة ستقوم على المقارنه كلمة كلمة اذا كانت الرسالة عادية وبها كلمات كالـ سبام ستعامل انها مزعجة

# الموديل الآن سيقرأ كل كلمة وحدها، وأيضاً كل كلمتين متتاليتين
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

X_train_count = vectorizer.fit_transform(X_train)

# تدريب الموديل
model = MultinomialNB()
model.fit(X_train_count, y_train)

# قياس الدقة
X_test_count = vectorizer.transform(X_test)
accuracy = model.score(X_test_count, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 94.44%


# الان سنختبر الموديل بداتا جديده

In [10]:
# 1. قائمة برسائل جديدة تماماً لم يراها الموديل من قبل
my_messages = [
    "Hey friend, I found a free course for a limited time with a prize!", # رسالة الصديق (مخادعة للموديل)
    "Congratulations! You won a 1000$ gift card, call now to claim",   # رسالة سبام واضحة
    "Are we meeting for lunch tomorrow?"                              # رسالة عادية واضحة
]

# 2. تنظيف الرسائل الجديدة بنفس الطريقة
cleaned_messages = [clean_text(m) for m in my_messages]

# 3. تحويلها لأرقام باستخدام نفس الـ vectorizer (مهم جداً عدم استخدام fit مرة أخرى)
my_samples_count = vectorizer.transform(cleaned_messages)

# 4. التوقع
predictions = model.predict(my_samples_count)

# طباعة النتائج
for msg, pred in zip(my_messages, predictions):
    status = "SPAM" if pred == 1 else "NORMAL (HAM)"
    print(f"Message: {msg}\nResult: {status}\n{'-'*20}")

Message: Hey friend, I found a free course for a limited time with a prize!
Result: NORMAL (HAM)
--------------------
Message: Congratulations! You won a 1000$ gift card, call now to claim
Result: SPAM
--------------------
Message: Are we meeting for lunch tomorrow?
Result: NORMAL (HAM)
--------------------


# كيف نحفظ هذا الموديل في ملف خارجي لكي نستخدمه في تطبيق أو موقع دون الحاجة لإعادة تدريبه في كل مرة

In [11]:
import joblib

# حفظ الموديل
joblib.dump(model, 'spam_detector_model.pkl')

# حفظ الـ Vectorizer (ضروري جداً لاستخدامه في التحويل لاحقاً)
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print("Model and Vectorizer saved successfully!")

Model and Vectorizer saved successfully!
